In [12]:
from sklearn.datasets import load_diabetes
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import time
import random

In [2]:
X,y = load_diabetes(return_X_y = True)
print(X.shape)
print(y.shape)

(442, 10)
(442,)


In [3]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)

In [4]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [5]:
print(reg.coef_)
print(reg.intercept_)

[  -9.15865318 -205.45432163  516.69374454  340.61999905 -895.5520019
  561.22067904  153.89310954  126.73139688  861.12700152   52.42112238]
151.88331005254167


In [6]:
y_pred = reg.predict(X_test)
r2_score(y_test,y_pred)

0.4399338661568968

### Let's create our own Mini Batch Gradient Descent Class

In [43]:
# Here the self.intercept_ is Beta 0 and self.coef_ are other parameters like Beta 1,2,...,n

class MGDRegressor:

  def __init__(self, batch_size,learning_rate=0.01, epochs=100): # Changed default lr to be safer
    self.coef_ = None
    self.intercept_ = None
    self.lr = learning_rate
    self.epochs = epochs
    self.batch_size = batch_size

  def fit(self, X_train, y_train):
    # Initialize your coefs
    self.intercept_ = 0
    self.coef_ = np.ones(X_train.shape[1])

    for i in range(self.epochs):
        for j in range(int(X_train.shape[0]/self.batch_size)):
            idx = random.sample(range(X_train.shape[0]),self.batch_size)
            # Let's calculate the 'y_hat' for the idx
            y_hat = np.dot(X_train[idx], self.coef_) + self.intercept_
            intercept_der = -2 * np.mean(y_train[idx] - y_hat)  # there is no np.mean() because we are dealing with the single row
            # now update the coef_ value
            self.intercept_ = self.intercept_ - (self.lr * intercept_der)
            # now coef_
            coef_der = -2 * np.dot((y_train[idx] - y_hat), X_train[idx]) 
            self.coef_ = self.coef_ - (self.lr * coef_der)

    print(f"Final Intercept: {self.intercept_}")
    print(f"Final Coefficients: {self.coef_}")

  def predict(self, X_test):
    return np.dot(X_test, self.coef_) + self.intercept_

In [60]:
mgd = MGDRegressor(batch_size = int(X_train.shape[0]/10),learning_rate = 0.01,epochs = 100)

In [61]:
mgd.fit(X_train, y_train)

Final Intercept: 151.82160207008934
Final Coefficients: [  23.00584965 -132.95280887  455.01423445  298.62592953  -25.61416978
  -90.71006783 -191.19313596  112.06407347  398.59488862  111.89758558]


In [62]:
y_pred = mgd.predict(X_test)

In [63]:
r2_score(y_test,y_pred)

0.45432827873828996

## General Way Of using Mini Batch GD

In [64]:
from sklearn.linear_model import SGDRegressor

In [69]:
sgd = SGDRegressor(learning_rate = 'constant',eta0 = 0.2)

In [70]:
batch_size = 35
for i in range(100):
    idx = random.sample(range(X_train.shape[0]),batch_size)
    sgd.partial_fit(X_train[idx], y_train[idx])

In [71]:
sgd.coef_

array([  39.53159028, -134.53344035,  439.11434419,  305.45679481,
        -33.40712889,  -85.54078288, -202.95260426,  111.95780632,
        389.78196825,  124.91698792])

In [72]:
sgd.intercept_

array([153.64813535])

In [73]:
y_pred = sgd.predict(X_test)

In [74]:
r2_score(y_test,y_pred)

0.45320003877630954